In [2]:
from urllib.error import HTTPError
from IPython.display import display
from pprint import pprint
import netCDF4
import ipywidgets as widgets
import urllib.request
import xml.etree.ElementTree as ET
import re

DATA_URLS = {"SPL1AP":"http://n5eil01u.ecs.nsidc.org/opendap/SMAP/SPL1AP.002/",
             "SPL1BTB 3":"http://n5eil01u.ecs.nsidc.org/opendap/SMAP/SPL1BTB.003/",
             "SPL1BTB 2":"http://n5eil01u.ecs.nsidc.org/opendap/SMAP/SPL1BTB.002/",
             "SPL1CTB 2":"http://n5eil01u.ecs.nsidc.org/opendap/SMAP/SPL1CTB.002/",
             "SPL1CTB 3":"http://n5eil01u.ecs.nsidc.org/opendap/SMAP/SPL1CTB.003/",
             "SPL2SMA 2":"http://n5eil01u.ecs.nsidc.org/opendap/SMAP/SPL2SMA.002/",
             "SPL2SMA 3":"http://n5eil01u.ecs.nsidc.org/opendap/SMAP/SPL2SMA.003/"}

SMAP_DATE_RANGE = [2015, 2016]

# Set up and display the widgets.
data_dropdown = widgets.Dropdown(options=DATA_URLS, value=DATA_URLS["SPL1AP"], description="Avaliable Data")
datasets_dropdown = widgets.Dropdown()
year_slider = widgets.IntSlider(min=SMAP_DATE_RANGE[0], max=SMAP_DATE_RANGE[1], step=1, value=0, description="Year")
month_slider = widgets.IntSlider(min=1, max=12, step=1, value=0, description="Month")
day_slider = widgets.IntSlider(min=1, max=31, step=1, value=0, description="Day")
hour_slider = widgets.IntSlider(min=0, max=24, step=1, value=0, description="Hours")
minutes_slider = widgets.IntSlider(min=0, max=60, step=1, value=0, description="Minutes")
seconds_slider = widgets.IntSlider(min=0, max=60, step=1, value=0, description="Seconds")
button = widgets.Button(description="Submit")
time_options_buttons = widgets.ToggleButtons(description="Display datasets that were recorded",
                                             options={'before this time' : 0, 'after this time' : 1})
display(data_dropdown)
display(year_slider)
display(month_slider)
display(day_slider)
display(time_options_buttons)
display(hour_slider)
display(button)

def get_current_data_selection():
    return data_dropdown.value

def retrieve_dataset(b):
    file_name = get_catalog_file_at_time(get_year_month_day())
    
    if 'file_name' in locals():
        #Parse the file as xml with ElementTree. 
        xml = open(file_name)
        tree = ET.parse(xml)
        root = tree.getroot()
        av_datasets = []

        #Loop through the catalog's "data" elements
        for child in root[2]:
            #If the file is an .h5 file, and not a .h5.iso.xml, then store its name.
            match = re.search(r"\.h5\Z", child.attrib["name"])
            if match:
                dataset_time = get_dataset_time(child.attrib["name"])
                user_time = get_user_time()
                if compare_times(dataset_time, user_time, time_options_buttons.value):
                    av_datasets.append(child.attrib["name"])
        
        #Print the matching dataset names.
        make_dropdown(av_datasets)

def get_user_time():
    return [hour_slider.value, minutes_slider.value, seconds_slider.value]
        
def get_dataset_time(dataset_name):
    time_string = re.search(r"T\d+",dataset_name).group(0)[1:]
    return [time_string[0:2], time_string[2:4], time_string[4:6]]

def compare_times(time1, time2, mode):
    hrs1 = int(time1[0])
    hrs2 = int(time2[0])
    if mode == 0:
        # Return true if time 1 occurs before time 2
        if hrs1 > hrs2:
            return False
        return True
    else:
        # Return true if time 1 occurs after time 2
        if hrs1 < hrs2:
            return False
        return True
        
def get_catalog_file_at_time(date):
    date_dotted, date_underscore = make_date_formats(date)
    try:
        #Retrieve the catalog file for the formed year
        file_name, headers = urllib.request.urlretrieve(get_current_data_selection()
                                                        +date_dotted+"/catalog.xml", 
                                                        "catalog_"+date_underscore+".xml")
        return file_name
    
    except HTTPError:
        #Returns if the url is improperly formatted or the date doesn't exist.
        print("Can't find data for " + date_dotted)
        
def get_dataset_at_time(dataset, date):
    date_dotted, date_underscore = make_date_formats(date)
    url = (get_current_data_selection()
           +date_dotted+"/"+dataset)
    return netCDF4.Dataset(url)
        
def get_year_month_day():
    return [year_slider.value, month_slider.value, day_slider.value]
        
def make_date_formats(date):
    #Create the date in a dotted and underscored format, as appears in the url
    date_dotted = str(date[0]) + "." + '{0:02d}'.format(date[1]) + "." + '{0:02d}'.format(date[2])
    date_underscore = str(date[0]) + "_" + str(date[1]) + "_" + str(date[2])
    return (date_dotted, date_underscore)

def handle_dropdown(b):
    selected_dataset_name = b["new"]
    dataset = get_dataset_at_time(selected_dataset_name, get_year_month_day())
    print("Variables in this dataset:\n")
    print(dataset.variables)
        
def make_dropdown(datasets_list):
    datasets_dropdown = widgets.Dropdown(options=datasets_list, value=datasets_list[0], description="Avaliable Datasets")
    display(datasets_dropdown)
    datasets_dropdown.observe(handle_dropdown, "value")
    
button.on_click(retrieve_dataset)